<a href="https://colab.research.google.com/github/oAlanSchmitt/CatalogAI/blob/main/CatalogAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTALAÇÃO DO SDK

In [1]:
!pip install -q -U google-generativeai

IMPORTANDO BIBLIOTECAS

In [28]:
import pathlib
import textwrap
import json
from IPython.display import display
from IPython.display import Markdown
import google.generativeai as genai

CONFIGURAÇÃO DA API_KEY

In [4]:
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

CONFIGURAÇÕS DO MODELO

In [35]:
# --- Configuração do modelo ---
generation_config = {"candidate_count": 1, "temperature": 0.7}

# É altamente recomendável manter os filtros de segurança ativados!
safety_settings = {
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

# Inicializando o modelo
model = genai.GenerativeModel(
    model_name='gemini-1.5-pro-latest',
    generation_config=generation_config,
    safety_settings=safety_settings,
)

# --- Instruções do sistema ---

# Instrução para identificar o gênero
identificar_genero_instruction = """
Você é um especialista em filmes, animes e séries.

Sua tarefa é identificar o gênero predominante em uma lista de títulos.

Responda apenas com o nome do gênero.

Por exemplo:

Títulos: The Walking Dead, Guerra Mundial Z, Zumbilândia
Gênero: Terror
"""

# Instrução para recomendar títulos
recomendar_instruction = """
Você é um assistente especializado em recomendar animes, filmes e séries.

Siga estas regras:

* **Recomendações únicas:** Nunca sugira o mesmo título que o usuário forneceu como entrada.
* **Diversidade:** Recomende um filme, um anime e uma série, um de cada.
* **Formato da resposta:** Separe as recomendações com "===" e use o seguinte formato para cada recomendação:
    * Tipo: [Filme, Anime ou Série]
    * Título: [Título]
    * Sinopse: [Breve sinopse]
    * Chances de Você Gostar: [Alta, Média ou Baixa]

Exemplo:

Informação: The Walking Dead, Guerra Mundial Z, Zumbilândia, John Wick, Mad Max
Resposta:
===
Tipo: Filme
Título: Army of the Dead
Sinopse: Um grupo de mercenários planeja um assalto a um cassino em Las Vegas durante um surto de zumbis.
Chances de Você Gostar: Alta
===
Tipo: Anime
Título: Highschool of the Dead
Sinopse: Um grupo de estudantes precisa lutar para sobreviver a um apocalipse zumbi.
Chances de Você Gostar: Média
===
Tipo: Série
Título: Kingdom
Sinopse: Um príncipe coreano precisa enfrentar uma misteriosa praga zumbi que está devastando o reino.
Chances de Você Gostar: Alta
"""

BASE DO MODELO

In [ ]:
chat = model.start_chat(history=[])

# --- Interação com o usuário ---
display(Markdown("##  ✨ Bem-vindo ao CatalogAI! ✨"))
display(Markdown("Sou seu assistente para descobrir novos filmes 🎬, animes 🏯 e séries 📺! "))
display(Markdown("Vou usar meu conhecimento para te recomendar títulos que combinem com o seu gosto! 🧠💡"))
display(Markdown(""))
display(Markdown("Conte-me sobre 5 títulos que você curte 💖 (pode ser qualquer tipo! 😄):"))
display(Markdown(""))
display(Markdown("### Por exemplo: "))
display(Markdown("- Matrix"))
display(Markdown("- Attack on Titan"))
display(Markdown("- Stranger Things"))

# Coleta de 5 títulos
titulos = []
for i in range(5):
    titulo = input(f"Título {i+1}: ")
    titulos.append(titulo)

# --- Identificação do gênero ---
prompt_genero = f"Títulos: {', '.join(titulos)}\nGênero: "
response_genero = chat.send_message(identificar_genero_instruction + prompt_genero)
genero = response_genero.text.strip()

# --- Interação personalizada ---
display(Markdown(f"Ah, então você curte {genero}! 🤩 Saca só essas recomendações:"))

# --- Loop de recomendações ---
while True:
    # --- Recomendações ---
    prompt_recomendacoes = f"Informação: {', '.join(titulos)}.\nResposta:\n"
    response_recomendacoes = chat.send_message(recomendar_instruction + prompt_recomendacoes)

    # --- Processamento e exibição da resposta ---
    try:
        recomendacoes = response_recomendacoes.text.strip().split("===")

        for recomendacao in recomendacoes:
            if recomendacao.strip():  # Ignora blocos vazios
                linhas = recomendacao.strip().split("\n")
                tipo = linhas[0].split(":")[1].strip()
                titulo = linhas[1].split(":")[1].strip()
                sinopse = linhas[2].split(":")[1].strip()
                chances = linhas[3].split(":")[1].strip()

                display(Markdown(f""))
                display(Markdown(f"## **{tipo}: {titulo}**"))
                display(Markdown(f"_{sinopse}_"))
                display(Markdown(f"**Chances de Você Gostar:** {chances}"))
                display(Markdown(f""))

    except Exception as e:
        print(f"Erro ao processar a resposta: {e}")

    # --- Perguntar ao usuário se deseja outra recomendação ---
    outra_recomendacao = input("Deseja outra recomendação? (s/n): ").lower()
    if outra_recomendacao != 's':
        break

##  ✨ Bem-vindo ao CatalogAI! ✨

Sou seu assistente para descobrir novos filmes 🎬, animes 🏯 e séries 📺! 

Vou usar meu conhecimento para te recomendar títulos que combinem com o seu gosto! 🧠💡

Conte-me sobre 5 títulos que você curte 💖 (pode ser qualquer tipo! 😄):

### Por exemplo: 

...

_Em um futuro distópico, um sistema prevê a probabilidade de uma pessoa cometer crimes, e a polícia usa essa tecnologia para deter criminosos em potencial._

**Chances de Você Gostar:** Média

## **Série: The Expanse**

_Em um futuro onde a humanidade colonizou o sistema solar, um detetive e um capitão de nave espacial se unem para investigar uma conspiração que ameaça a paz._

**Chances de Você Gostar:** Alta